In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
data = pd.read_csv('/content/Crop_recommendation.csv')

X = data[['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall']]
y = data['label']

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)  # Convert labels to one-hot encoded format

In [ ]:
# Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42, stratify=y_encoded)

# Build the model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(y_categorical.shape[1], activation='softmax')  # Output layer with softmax for multi-class classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with validation split
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.0530 - loss: 4.0046 - val_accuracy: 0.0900 - val_loss: 3.4014
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1504 - loss: 3.2118 - val_accuracy: 0.3988 - val_loss: 2.3779
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3741 - loss: 2.4521 - val_accuracy: 0.5378 - val_loss: 1.8140
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4512 - loss: 2.0212 - val_accuracy: 0.5930 - val_loss: 1.4618
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5246 - loss: 1.6895 - val_accuracy: 0.6564 - val_loss: 1.1883
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5624 - loss: 1.4616 - val_accuracy: 0.6973 - val_loss: 1.0301
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5859 - loss: 1.3006 - val_accuracy: 0.7403 - val_loss: 0.9140
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6276 - loss: 1.1896 - val_accuracy: 0.7464 -

In [ ]:
# Predict on the test set
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [ ]:
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels_decoded = label_encoder.inverse_transform(y_test_labels)

In [ ]:
accuracy = accuracy_score(y_test_labels_decoded, y_pred_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 87.56%


In [ ]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_labels_decoded, y_pred_labels))


Confusion Matrix:
[[ 3  0  0 ...  0  0  0]
 [ 0  4  0 ...  0  0  0]
 [ 0  0  3 ...  0  0  0]
 ...
 [ 0  0  0 ... 20  0  0]
 [ 0  0  0 ...  0 19  0]
 [ 0  0  0 ...  0  0 20]]


In [ ]:
# Display classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test_labels_decoded, y_pred_labels))


Classification Report:
                   precision    recall  f1-score   support

           Almond       0.75      0.60      0.67         5
             Amla       0.50      0.80      0.62         5
           Barley       1.00      1.00      1.00         3
         Beetroot       1.00      0.83      0.91         6
      Bell Pepper       0.29      0.33      0.31         6
     Bitter Gourd       0.45      0.83      0.59         6
     Bottle Gourd       0.67      0.80      0.73         5
          Brinjal       0.00      0.00      0.00         3
          Cabbage       0.25      0.17      0.20         6
           Carrot       0.67      0.67      0.67         3
           Cashew       0.83      0.83      0.83         6
Cassava (Tapioca)       1.00      1.00      1.00         3
      Cauliflower       0.50      0.67      0.57         3
        Coriander       0.30      0.50      0.38         6
    Custard Apple       0.00      0.00      0.00         3
        Drumstick       0.75   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Function to predict crop based on user input
def predict_crop(model, scaler, label_encoder):
    # Take user input for soil features
    try:
        N = float(input("Enter Nitrogen content (N): "))
        P = float(input("Enter Phosphorus content (P): "))
        K = float(input("Enter Potassium content (K): "))
        temperature = float(input("Enter temperature (°C): "))
        humidity = float(input("Enter humidity (%): "))
        ph = float(input("Enter pH level: "))
        rainfall = float(input("Enter rainfall (mm): "))

        # Create a DataFrame for the input
        input_data = pd.DataFrame([[N, P, K, temperature, humidity, ph, rainfall]],
                                   columns=['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall'])

        # Scale the input features
        input_scaled = scaler.transform(input_data)

        # Predict using the model
        prediction_proba = model.predict(input_scaled)
        prediction = np.argmax(prediction_proba, axis=1)

        # Decode the predicted label
        predicted_label = label_encoder.inverse_transform(prediction)

        print(f"\nRecommended crop: {predicted_label[0]}")

    except ValueError:
        print("Please enter valid numerical values.")

# Example usage
predict_crop(model, scaler, label_encoder)

Enter Nitrogen content (N): 4
Enter Phosphorus content (P): 3
Enter Potassium content (K): 
Please enter valid numerical values.


In [ ]:
print(y.value_counts())
#number of unique values in the target variiable, i.e. the number of unique crops we can recommend

label
rice             100
grapes           100
maize            100
coffee           100
jute             100
                ... 
Custard Apple     15
Brinjal           15
Tamarind          15
Spinach           15
Potato            14
Name: count, Length: 62, dtype: int64
